In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 483.0MB 53.5MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x56320000 @  0x7f87fe88e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [0]:
!pip3 install torchvision

    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 2.0MB 9.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch

In [0]:
!pwd

In [0]:
!ls

data	  logs	      models		 __pycache__	 train_lstm.py
dataset   Main.ipynb  pretrained_models  raw		 utils.py
datasets  model.pth   processed		 train_drnet.py


In [0]:
cd drive/PyTorch/Motion_prediction

/content/drive/PyTorch/Motion_prediction


In [0]:
cd ..

/content


In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
import PIL.image

### Install progressbar

In [0]:
!pip3 install progressbar

  Running setup.py bdist_wheel for progressbar ... - done
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


In [0]:
!python drive/PyTorch/starting_with_codelab/simple_gan.py

In [0]:
import numpy as np
import matplotlib.pyplot as plt

dataset_path='dataset/mnist_test_seq.npy'
motion_mnist=np.load(dataset_path)

In [0]:
a=motion_mnist[:,1,:,:]
a.shape
plt.ion()
for i in range(20):
  plt.figure(1); plt.clf()
  plt.imshow(a[i,:,:])
  plt.title('Number ' + str(i))
  plt.pause(.3)


### Train New GAN

In [0]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 5
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        #print('output size is:',fake_images.shape)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
            print('output size is:',outputs.shape)
            for i in range(4):
              plt.subplot(2,2,i+1)
              plt.imshow(np.reshape(fake_images.data.cpu().numpy()[i], (28, 28)), cmap='gray')
            plt.show()
    
    # Save real images
    #if (epoch+1) == 1:
    #    images = images.reshape(images.size(0), 1, 28, 28)
    #    save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    print('fake_images.size')
    save_image(fake_images, os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

### Train DR Net

In [0]:
# lam nhu trong paper

import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import itertools
import progressbar

from data.moving_mnist import MovingMNIST
import matplotlib.pyplot as plt


## hyper parameter

lr=0.002
beta1=0.9
batch_size=100
log_dir='logs'

data_root_x=''
optimizer='adam'
niter=200
seed=1
epoch_size=600
content_dim=128
pose_dim=5
image_width=64
channels=3
dataset='moving_mnist'

max_step=20
sd_weight=0.1
sd_nf=100
content_model='dcgan'
pose_model='dcgan'
data_threads=0
normalize='store_true'
data_type='drnet'




name = 'content_model=%s\npose_model=%s\ncontent_dim=%d\npose_dim=%d\nmax_step=%d\nsd_weight=%.3f\nlr=%.3f\nsd_nf=%d\nnormalize=%s' % (content_model, pose_model, content_dim, pose_dim, max_step, sd_weight, lr, sd_nf, normalize)
log_dir = '%s/%s%dx%d/%s' % (log_dir, dataset, image_width, image_width, 'xxx')

os.makedirs('%s/rec/' % log_dir, exist_ok=True)
os.makedirs('%s/analogy/' % log_dir, exist_ok=True)

print(name)

print(log_dir)

print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
dtype = torch.cuda.FloatTensor

if image_width == 64:
    import models.resnet_64 as resnet_models
    import models.dcgan_64 as dcgan_models
    import models.dcgan_unet_64 as dcgan_unet_models
    import models.vgg_unet_64 as vgg_unet_models

if content_model == 'dcgan':
    netEC = dcgan_models.content_encoder(content_dim, channels)
    netD = dcgan_models.decoder(content_dim, pose_dim, channels)

else:
    raise ValueError('Unknown content model: %s' % content_model)

if pose_model == 'dcgan':
    netEP = dcgan_models.pose_encoder(pose_dim, channels, normalize=normalize)
else:
    raise ValueError('Unknown pose model: %s' % pose_model)   
    
       
import models.classifiers as classifiers
netC = classifiers.scene_discriminator(pose_dim, sd_nf)

netEC.apply(utils.init_weights)
netEP.apply(utils.init_weights)
netD.apply(utils.init_weights)
netC.apply(utils.init_weights)
    
  
# ---------------- optimizers ----------------

if optimizer == 'adam':
    optimizer = optim.Adam
else:
  raise ValueError('Unknown optimizer: %s' % optimizer)

optimizerC = optimizer(netC.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerEC = optimizer(netEC.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerEP = optimizer(netEP.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optimizer(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    
# --------- loss functions ------------------------------------
mse_criterion = nn.MSELoss()
bce_criterion = nn.BCELoss()

# --------- transfer to gpu ------------------------------------
netEP.cuda()
netEC.cuda()
netD.cuda()
netC.cuda()
mse_criterion.cuda()
bce_criterion.cuda()


# --------- load a dataset ------------------------------------

train_data, test_data = utils.load_dataset(dataset,data_root_x,max_step,image_width,data_type)

train_loader = DataLoader(train_data,
                          num_workers=data_threads,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          pin_memory=True)
test_loader = DataLoader(test_data,
                         num_workers=data_threads,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True,
                         pin_memory=True)


print(vars(train_loader))


def get_training_batch():
    while True:
        for sequence in train_loader:
            batch = utils.normalize_data(dataset,channels, dtype, sequence)
            yield batch
training_batch_generator = get_training_batch()

def get_testing_batch():
    while True:
        for sequence in test_loader:
            
            batch = utils.normalize_data(dataset,channels, dtype, sequence)
            yield batch
testing_batch_generator = get_testing_batch()


# --------- plotting funtions ------------------------------------

def plot_rec(x, epoch):
      x_c = x[0]
      x_p = x[np.random.randint(1, max_step)]

      h_c = netEC(x_c)
      h_p = netEP(x_p)
      
      h_motion=torch.cuda.FloatTensor(100,5,1,1).fill_(0)
      h_content=torch.cuda.FloatTensor(100,128,1,1).fill_(0)  
#       h4_c=torch.cuda.FloatTensor(100,512,4,4).fill_(0) 
#       h3_c=torch.cuda.FloatTensor(100,256,8,8).fill_(0) 
#       h2_c=torch.cuda.FloatTensor(100,128,16,16).fill_(0) 
#       h1_c=torch.cuda.FloatTensor(100,64,32,32).fill_(0) 
      
#       h_content=[h5_c,[h1_c,h2_c,h3_c,h4_c]]
      
      rec = netD([h_c, h_p])
      rec_motion = netD([h_content, h_p])
      rec_content = netD([h_c, h_motion])
      

      rec_=rec.cpu().detach().numpy()
      rec_motion_= rec_motion.cpu().detach().numpy()
      rec_content_ = rec_content.cpu().detach().numpy()
      
      
      
#       print(type(h_c))
      
#       print('len1:',len(h_c))
#       print('len2:',len(h_c[0]))
#       print('len3:',len(h_c[0][0]))
#       print('len4:',len(h_c[0][0][0]))
#       print('len5:',len(h_c[0][0][0][0]))
    
    
#       print('len1:',len(h_p))
#       print('len2:',len(h_p[0]))
#       print('len3:',len(h_p[0][0]))
#       print('len3:',len(h_p[0][0][0]))
      
    
    
      
#       print(type(h_p))
#       print("motion size; ",h_p.shape)
      
      
      #### show output size cua 1 model
      
#       h_c_=h_c
#       h_p_=h_p      
#       print('size of motion output: ', h_p_.size())
#       print('size of content output: ', h_c_.size())
      
  
      print('reconstruction from motion and content ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_[i],(1,2,0)))
      plt.show()
      
      print('reconstruction from motion ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_motion_[i],(1,2,0)))
      plt.show()
      
      print('reconstruction from content ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_content_[i],(1,2,0)))
      plt.show()
                   

        
        
def plot_analogy(x, epoch):
    x_c = x[0]

    h_c = netEC(x_c)
    nrow = 10
    row_sz = max_step 
    to_plot = []
    row = [xi[0].data for xi in x]
    zeros = torch.zeros(channels, image_width, image_width)
    to_plot.append([zeros] + row)
    for i in range(nrow):
        to_plot.append([x[0][i].data])

    for j in range(0, row_sz):
        h_p = netEP(x[j]).data
        for i in range(nrow):
            h_p[i] = h_p[0]
        rec = netD([h_c, Variable(h_p)])
        for i in range(nrow):
            to_plot[i+1].append(rec[i].data.clone())

    fname = '%s/analogy/%d.png' % (log_dir, epoch) 
    utils.save_tensors_image(fname, to_plot)

   
    

# --------- training funtions ------------------------------------
def train(x):
    netEP.zero_grad()
    netEC.zero_grad()
    netD.zero_grad()

    x_c1 = x[np.random.randint(4, max_step)]
    x_c2 = x[0]
    x_p1 = x[np.random.randint(4, max_step)]
    x_p2 = x[3]

    h_c1 = netEC(x_c1)
    h_c2 = netEC(x_c2)[0].detach() if content_model[-4:] == 'unet' else netEC(x_c2).detach() # used as target for sim loss
    h_p1 = netEP(x_p1) # used for scene discriminator
    h_p2 = netEP(x_p2).detach()


    # similarity loss: ||h_c1 - h_c2||
    sim_loss = mse_criterion(h_c1[0] if content_model[-4:] == 'unet' else h_c1, h_c2)


    # reconstruction loss: ||D(h_c1, h_p1), x_p1|| 
    rec = netD([h_c1, h_p1])
    rec_loss = mse_criterion(rec, x_p1)

    # scene discriminator loss: maximize entropy of output
    target = torch.cuda.FloatTensor(batch_size, 1).fill_(0.5)
    out = netC([h_p1, h_p2])
    sd_loss = bce_criterion(out, Variable(target))

    # full loss
    loss = sim_loss + rec_loss + sd_weight*sd_loss
    loss.backward()

    optimizerEC.step()
    optimizerEP.step()
    optimizerD.step()

    return sim_loss.data.cpu().numpy(),rec_loss.data.cpu().numpy(), sd_loss.data.cpu().numpy() 

  
def train_scene_discriminator(x):
  
    netC.zero_grad() ############# Sets gradients of all model parameters to zero.

    target = torch.cuda.FloatTensor(batch_size, 1)

    x1 = x[0]
    x2 = x[9]
    
    h_p1 = netEP(x1).detach()
    h_p2 = netEP(x2).detach()

    half = int(batch_size/2)
    rp = torch.randperm(half).cuda()
    h_p2[:half] = h_p2[rp]
    target[:half] = 1
    target[half:] = 0

    out = netC([h_p1, h_p2])
    bce = bce_criterion(out, Variable(target))

    bce.backward()
    optimizerC.step()

    acc =out[:half].gt(0.5).sum() + out[half:].le(0.5).sum()
    return bce.data.cpu().numpy(), acc.data.cpu().numpy()/batch_size


  
# --------- training loop ------------------------------------
for epoch in range(niter):
  
    netEP.train()
    netEC.train()
    netD.train()
    netC.train()
    
    epoch_sim_loss, epoch_rec_loss, epoch_sd_loss, epoch_sd_acc = 0, 0, 0, 0

    progress = progressbar.ProgressBar(maxval=epoch_size).start()
    for i in range(epoch_size):
        progress.update(i+1)
        x = next(training_batch_generator)

        # train scene discriminator
        _, sd_acc = train_scene_discriminator(x)
#         epoch_sd_loss += sd_loss
        epoch_sd_acc += sd_acc

        # train main model
        sim_loss, rec_loss, sd_loss = train(x)
        epoch_sd_loss += sd_loss
        epoch_sim_loss += sim_loss
        epoch_rec_loss += rec_loss
        
#         if (epoch_size+1) % 2 == 0:
#           print('type of x: ',type(x))
#           print('size of x: ', len(x))
          
#           print('size of x: ', len(x[1]))
#           print('size of x: ', len(x[1][1]))
#           print('size of x: ', len(x[1][1][1]))
          


    progress.finish()
    utils.clear_progressbar()

    netEP.eval()
    netEC.eval()
    netD.eval()
    # plot some stuff
    x = next(testing_batch_generator)
    plot_rec(x, epoch)
    

    print('[%02d] rec loss: %.4f | sim loss: %.6f | scene disc acc: %.3f%% | sd_loss: %.4f (%d)' % (epoch, epoch_rec_loss/epoch_size, epoch_sim_loss/epoch_size, 100*epoch_sd_acc/epoch_size, epoch_sd_loss/epoch_size, epoch*epoch_size))

    # save the model
    
    if (epoch+1) % 20 ==0:
      state = {'epoch': epoch + 1, '
               
               ': netD.state_dict(), 'state_dict_netEP': netEP.state_dict(), 'state_dict_netEC': netEC.state_dict(), 'state_dict_netC': netC.state_dict(),
             'optimizerC': optimizerC.state_dict(), 'optimizerD': optimizerD.state_dict(), 'optimizerEP': optimizerEP.state_dict(), 'optimizerEC': optimizerEC.state_dict(),}
      torch.save(state, '%s/model_state_dcgan_like_paper_epoch_%d.pth' % (log_dir,epoch+1))
    
    
 

    
#     torch.save({
#         'netD': netD,
#         'netEP': netEP,
#         'netEC': netEC,},
#         '%s/model.pth' % log_dir)

Output hidden; open in https://colab.research.google.com to view.

### Load model and continue training

In [0]:
import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import itertools
import progressbar

from data.moving_mnist import MovingMNIST
import matplotlib.pyplot as plt


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## hyper parameter

lr=0.002
beta1=0.9
batch_size=100
log_dir='logs'

data_root_x=''
optimizer='adam'
niter=260
seed=1
epoch_size=600
content_dim=128
pose_dim=10
image_width=64
channels=3
dataset='moving_mnist'

max_step=20
sd_weight=0.05
sd_nf=100
content_model='dcgan_unet'
pose_model='dcgan'
data_threads=0
normalize='store_true'
data_type='drnet'




name = 'content_model=%s\npose_model=%s\ncontent_dim=%d\npose_dim=%d\nmax_step=%d\nsd_weight=%.3f\nlr=%.3f\nsd_nf=%d\nnormalize=%s' % (content_model, pose_model, content_dim, pose_dim, max_step, sd_weight, lr, sd_nf, normalize)
log_dir = '%s/%s%dx%d/%s' % (log_dir, dataset, image_width, image_width, 'xxx')

# os.makedirs('%s/rec/' % log_dir, exist_ok=True)
# os.makedirs('%s/analogy/' % log_dir, exist_ok=True)

print(name)

print(log_dir)

print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
dtype = torch.cuda.FloatTensor

if image_width == 64:
    import models.resnet_64 as resnet_models
    import models.dcgan_64 as dcgan_models
    import models.dcgan_unet_64 as dcgan_unet_models
    import models.vgg_unet_64 as vgg_unet_models

if content_model == 'dcgan_unet':
    netEC = dcgan_unet_models.content_encoder(content_dim, channels)
    netD = dcgan_unet_models.decoder(content_dim, pose_dim, channels)

else:
    raise ValueError('Unknown content model: %s' % content_model)

if pose_model == 'dcgan':
    netEP = dcgan_models.pose_encoder(pose_dim, channels, normalize=normalize)
else:
    raise ValueError('Unknown pose model: %s' % pose_model)   
    
       
import models.classifiers as classifiers
netC = classifiers.scene_discriminator(pose_dim, sd_nf)

# netEC.apply(utils.init_weights)
# netEP.apply(utils.init_weights)
# netD.apply(utils.init_weights)
# netC.apply(utils.init_weights)
    
  
# ---------------- optimizers ----------------

if optimizer == 'adam':
    optimizer = optim.Adam
else:
  raise ValueError('Unknown optimizer: %s' % optimizer)

optimizerC = optimizer(netC.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerEC = optimizer(netEC.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerEP = optimizer(netEP.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optimizer(netD.parameters(), lr=lr, betas=(beta1, 0.999))
  
# -------------------- load model, optimizer and epoch ---------------  

checkpoint = torch.load('%s/model_state_epoch_100.pth' % log_dir)

start_epoch = checkpoint['epoch']

netD.load_state_dict(checkpoint['state_dict_netD'])
optimizerD.load_state_dict(checkpoint['optimizerD'])

netC.load_state_dict(checkpoint['state_dict_netC'])
optimizerC.load_state_dict(checkpoint['optimizerC'])

netEP.load_state_dict(checkpoint['state_dict_netEP'])
optimizerEP.load_state_dict(checkpoint['optimizerEP'])

netEC.load_state_dict(checkpoint['state_dict_netEC'])
optimizerEC.load_state_dict(checkpoint['optimizerEC'])

    
# --------- loss functions ------------------------------------
mse_criterion = nn.MSELoss()
bce_criterion = nn.BCELoss()

# --------- transfer to gpu ------------------------------------
netEP.cuda()
netEC.cuda()
netD.cuda()
netC.cuda()
mse_criterion.cuda()
bce_criterion.cuda()

#-----------transfer optimizer to gpu

# now individually transfer the optimizer parts...
for state in optimizerD.state.values():
  for k, v in state.items():
    if isinstance(v, torch.Tensor):
      state[k] = v.to(device)
      
for state in optimizerC.state.values():
  for k, v in state.items():
    if isinstance(v, torch.Tensor):
      state[k] = v.to(device)      
      
for state in optimizerEP.state.values():
  for k, v in state.items():
    if isinstance(v, torch.Tensor):
      state[k] = v.to(device)     
      

for state in optimizerEC.state.values():
  for k, v in state.items():
    if isinstance(v, torch.Tensor):
      state[k] = v.to(device)      
      

# --------- load a dataset ------------------------------------

train_data, test_data = utils.load_dataset(dataset,data_root_x,max_step,image_width,data_type)

train_loader = DataLoader(train_data,
                          num_workers=data_threads,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          pin_memory=True)
test_loader = DataLoader(test_data,
                         num_workers=data_threads,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True,
                         pin_memory=True)


print(vars(train_loader))


def get_training_batch():
    while True:
        for sequence in train_loader:
            batch = utils.normalize_data(dataset,channels, dtype, sequence)
            yield batch
training_batch_generator = get_training_batch()

def get_testing_batch():
    while True:
        for sequence in test_loader:
            
            batch = utils.normalize_data(dataset,channels, dtype, sequence)
            yield batch
testing_batch_generator = get_testing_batch()


# --------- plotting funtions ------------------------------------

def plot_rec(x, epoch):
      x_c = x[0]
      x_p = x[np.random.randint(1, max_step)]

      h_c = netEC(x_c)
      h_p = netEP(x_p)
      
      h_motion=torch.cuda.FloatTensor(100,10,1,1).fill_(0)
      h5_c=torch.cuda.FloatTensor(100,128,1,1).fill_(0)  
      h4_c=torch.cuda.FloatTensor(100,512,4,4).fill_(0) 
      h3_c=torch.cuda.FloatTensor(100,256,8,8).fill_(0) 
      h2_c=torch.cuda.FloatTensor(100,128,16,16).fill_(0) 
      h1_c=torch.cuda.FloatTensor(100,64,32,32).fill_(0) 
      
      h_content=[h5_c,[h1_c,h2_c,h3_c,h4_c]]
      
      rec = netD([h_c, h_p])
      rec_motion = netD([h_content, h_p])
      rec_content = netD([h_c, h_motion])
      

      rec_=rec.cpu().detach().numpy()
      rec_motion_= rec_motion.cpu().detach().numpy()
      rec_content_ = rec_content.cpu().detach().numpy()
      
      
      
#       print(type(h_c))
      
#       print('len1:',len(h_c))
#       print('len2:',len(h_c[0]))
#       print('len3:',len(h_c[0][0]))
#       print('len4:',len(h_c[0][0][0]))
#       print('len5:',len(h_c[0][0][0][0]))
    
    
#       print('len1:',len(h_p))
#       print('len2:',len(h_p[0]))
#       print('len3:',len(h_p[0][0]))
#       print('len3:',len(h_p[0][0][0]))
      
    
    
      
#       print(type(h_p))
#       print("motion size; ",h_p.shape)
      
      
      #### show output size cua 1 model
      
#       h_c_=h_c
#       h_p_=h_p      
#       print('size of motion output: ', h_p_.size())
#       print('size of content output: ', h_c_.size())
      
  
      print('reconstruction from motion and content ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_[i],(1,2,0)))
      plt.show()
      
      print('reconstruction from motion ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_motion_[i],(1,2,0)))
      plt.show()
      
      print('reconstruction from content ')
      for i in range(2):
        plt.subplot(1,2,i+1)
        plt.imshow(np.transpose(rec_content_[i],(1,2,0)))
      plt.show()
                   

        
        
def plot_analogy(x, epoch):
    x_c = x[0]

    h_c = netEC(x_c)
    nrow = 10
    row_sz = max_step 
    to_plot = []
    row = [xi[0].data for xi in x]
    zeros = torch.zeros(channels, image_width, image_width)
    to_plot.append([zeros] + row)
    for i in range(nrow):
        to_plot.append([x[0][i].data])

    for j in range(0, row_sz):
        h_p = netEP(x[j]).data
        for i in range(nrow):
            h_p[i] = h_p[0]
        rec = netD([h_c, Variable(h_p)])
        for i in range(nrow):
            to_plot[i+1].append(rec[i].data.clone())

    fname = '%s/analogy/%d.png' % (log_dir, epoch) 
    utils.save_tensors_image(fname, to_plot)

   
    

# --------- training funtions ------------------------------------
def train(x):
    netEP.zero_grad()
    netEC.zero_grad()
    netD.zero_grad()

    x_c1 = x[np.random.randint(2, max_step)]
    x_c2 = x[0]
    x_p1 = x[np.random.randint(2, max_step)]
    x_p2 = x[1]

    h_c1 = netEC(x_c1)
    h_c2 = netEC(x_c2)[0].detach() if content_model[-4:] == 'unet' else netEC(x_c2).detach() # used as target for sim loss
    h_p1 = netEP(x_p1) # used for scene discriminator
    h_p2 = netEP(x_p2).detach()


    # similarity loss: ||h_c1 - h_c2||
    sim_loss = mse_criterion(h_c1[0] if content_model[-4:] == 'unet' else h_c1, h_c2)


    # reconstruction loss: ||D(h_c1, h_p1), x_p1|| 
    rec = netD([h_c1, h_p1])
    rec_loss = mse_criterion(rec, x_p1)

    # scene discriminator loss: maximize entropy of output
    target = torch.cuda.FloatTensor(batch_size, 1).fill_(0.5)
    out = netC([h_p1, h_p2])
    sd_loss = bce_criterion(out, Variable(target))

    # full loss
    loss = sim_loss + rec_loss + sd_weight*sd_loss
    loss.backward()

    optimizerEC.step()
    optimizerEP.step()
    optimizerD.step()

    return sim_loss.data.cpu().numpy(),rec_loss.data.cpu().numpy(), sd_loss.data.cpu().numpy()

  
def train_scene_discriminator(x):
  
    netC.zero_grad() ############# Sets gradients of all model parameters to zero.

    target = torch.cuda.FloatTensor(batch_size, 1)

    x1 = x[0]
    x2 = x[1]
    
    h_p1 = netEP(x1).detach()
    h_p2 = netEP(x2).detach()

    half = int(batch_size/2)
    rp = torch.randperm(half).cuda()
    h_p2[:half] = h_p2[rp]
    target[:half] = 1
    target[half:] = 0

    out = netC([h_p1, h_p2])
    bce = bce_criterion(out, Variable(target))

    bce.backward()
    optimizerC.step()

    acc =out[:half].gt(0.5).sum() + out[half:].le(0.5).sum()
    return bce.data.cpu().numpy(), acc.data.cpu().numpy()/batch_size


  
# --------- training loop ------------------------------------
for epoch in range(start_epoch, niter):
  
    netEP.train()
    netEC.train()
    netD.train()
    netC.train()
    
    epoch_sim_loss, epoch_rec_loss, epoch_sd_loss, epoch_sd_acc = 0, 0, 0, 0

    progress = progressbar.ProgressBar(maxval=epoch_size).start()
    for i in range(epoch_size):
        progress.update(i+1)
        x = next(training_batch_generator)

        # train scene discriminator
        _, sd_acc = train_scene_discriminator(x)
#         epoch_sd_loss += sd_loss
        epoch_sd_acc += sd_acc

        # train main model
        sim_loss, rec_loss, sd_loss = train(x)
        epoch_sim_loss += sim_loss
        epoch_rec_loss += rec_loss
        epoch_sd_loss += sd_loss
        
#         if (epoch_size+1) % 2 == 0:
#           print('type of x: ',type(x))
#           print('size of x: ', len(x))
          
#           print('size of x: ', len(x[1]))
#           print('size of x: ', len(x[1][1]))
#           print('size of x: ', len(x[1][1][1]))
          


    progress.finish()
    utils.clear_progressbar()

    netEP.eval()
    netEC.eval()
    netD.eval()
    # plot some stuff
    x = next(testing_batch_generator)
    plot_rec(x, epoch)
    

    print('[%02d] rec loss: %.4f | sim loss: %.6f | scene disc acc: %.3f%% | sd_loss:%.4f (%d)' % (epoch, epoch_rec_loss/epoch_size, epoch_sim_loss/epoch_size, 100*epoch_sd_acc/epoch_size, epoch_sd_loss/epoch_size, epoch*epoch_size*batch_size))

    # save the model
    
    if (epoch+1) % 20==0:
      state = {'epoch': epoch + 1, 'state_dict_netD': netD.state_dict(), 'state_dict_netEP': netEP.state_dict(), 'state_dict_netEC': netEC.state_dict(), 'state_dict_netC': netC.state_dict(),
             'optimizerC': optimizerC.state_dict(), 'optimizerD': optimizerD.state_dict(), 'optimizerEP': optimizerEP.state_dict(), 'optimizerEC': optimizerEC.state_dict(),}
      torch.save(state, '%s/model_state_epoch%d.pth' % (log_dir,epoch+1))
    
       
#     torch.save({
#         'netD': netD,
#         'netEP': netEP,
#         'netEC': netEC,},
#         '%s/model.pth' % log_dir)

### Save model for testing

In [0]:

import torch  
  
  
# save the model
torch.save({
        'netD': netD,
        'netEP': netEP,
        'netEC': netEC,},
        '%s/model.pth' % log_dir)

In [0]:
!ls

In [0]:
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
import random


# t=[[[1,2],[6,4],[8,9]], [3,4], [5,6]]

# print(len(t[0][1]))

# print("=============================")

# a=torch.randn(2,3)
# print(a)

# a_=torch.randn(2,3)
# print(a_)



# b=torch.cat([a,a_],1)
# print(b)

# print('===========================')
# c=torch.range(1,16)
# print(c)
# c_=c.view(-1,2)
# print(c_)

# print('============================')

# target = torch.cuda.FloatTensor(5, 1).fill_(0.5)
# print(target)
# print(Variable(target))
 
  
target_n =torch.cuda.FloatTensor(10,1)
print(target_n)






### Test_loading data

In [0]:
import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import itertools
import progressbar

from data.moving_mnist import MovingMNIST
import matplotlib.pyplot as plt


## hyper parameter

lr=0.002
beta1=0.5
batch_size=100
log_dir='logs'

data_root_x=''
optimizer='adam'
niter=200
seed=1
epoch_size=600
content_dim=128
pose_dim=10
image_width=64
channels=3
dataset='moving_mnist'
max_step=20
sd_weight=0.0001
sd_nf=100
content_model='dcgan_unet'
pose_model='dcgan'
data_threads=5
normalize='store_true'
data_type='drnet'




name = 'content_model=%s\npose_model=%s\ncontent_dim=%d\npose_dim=%d\nmax_step=%d\nsd_weight=%.3f\nlr=%.3f\nsd_nf=%d\nnormalize=%s' % (content_model, pose_model, content_dim, pose_dim, max_step, sd_weight, lr, sd_nf, normalize)
log_dir = '%s/%s%dx%d/%s' % (log_dir, dataset, image_width, image_width, 'xxx')

# os.makedirs('%s/rec/' % log_dir, exist_ok=True)
# os.makedirs('%s/analogy/' % log_dir, exist_ok=True)

print(name)

print(log_dir)

print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
dtype = torch.cuda.FloatTensor


# --------- load a dataset ------------------------------------

train_data = MovingMNIST(
                train=True,
                data_root=data_root_x,
                seq_len=max_step,
                image_size=image_width,
                num_digits=2)



# train_data, test_data = utils.load_dataset(dataset,data_root_x,max_step,image_width,data_type)

# train_loader = DataLoader(train_data,
#                           num_workers=data_threads,
#                           batch_size=batch_size,
#                           shuffle=True,
#                           drop_last=True,
#                           pin_memory=True)

In [0]:
!pip3 install scipy 